# Exercise 03
## Re-ID with Graph Neural Networks

In this exercise, we will first create an extension of the IoU- and ReID-based tracker we created in the previous exercise that will make it more robust to occlusions by allowing it to recover from missed detections.

We will then implement a Message Passing Network that we will use to learn features that combine position and appearance information. On these edge features, we then base our prediction on associations between past tracks and new detections.

<!-- TODO: adapt tasks --> 
Your tasks are the following:
- Adapt the track management scheme of our Tracker to allow it to recover from missed detections.
- Implement the Update Operation for the Message Passing Network to operate on bipartite graphs
- Implement the pairwise motion feature computation to obtain initial features for the edges of our Message Passing Network
- Train the Message Passing Network and improve your tracker's IDF1 score
- Use the Message Passing Network instead of the distance matrix in the Tracker

## Imports

This will setup your whole environment such that you can work with the rest of the notebook.

### General Imports

In [1]:
from pathlib import Path

import torch
from torch.utils.data import DataLoader

/home2/yehia.ahmed/SafeML_2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set up directory paths and (optionally) mount in Google Colab
If you work with google colab set the `USING_COLAB` variable to `True` and following cell to mount your gdrive.

In [2]:
USING_COLAB = False
USE_CPU = False
# Use the following lines if you want to use Google Colab
# We presume you created a folder "cv3dst" within your main drive folder, and put the exercise there.
# NOTE: terminate all other colab sessions that use GPU!
# NOTE 2: Make sure the correct exercise folder (e.g exercise_03) is given.

if USING_COLAB:
    from google.colab import drive
    import os

    gdrive_path='/content/gdrive/MyDrive/cv3dst/exercise_03'

    # This will mount your google drive under 'MyDrive'
    drive.mount('/content/gdrive', force_remount=True)
    # In order to access the files in this notebook we have to navigate to the correct folder
    os.chdir(gdrive_path)
    # Check manually if all files are present
    print(sorted(os.listdir()))
    root_dir = Path(gdrive_path).parent
else:
    root_dir = Path('/home2/yehia.ahmed/cv3/cv3dst')
# NOTE: Also adjust the root_dir in test/hungarian_tracking.py

dataset_dir = root_dir.joinpath("datasets")
output_dir = root_dir.joinpath('exercise_03', 'models')
output_dir.mkdir(parents=True, exist_ok=True)

device = torch.device('cuda') if torch.cuda.is_available() and not USE_CPU else torch.device('cpu')

### Exercise Specific Imports

In [3]:

from exercise_code import (
    MOT16Sequences,
    LongTrackTrainingDataset,
)

from exercise_code import (
    Hungarian_TrackerIoUReID,
    Longterm_Hungarian_TrackerIoUReID,
    MPN_Tracker,
    AssignmentSimilarityNet,
    train_assign_net_one_epoch,
    evaluate_assign_net,
)

from exercise_code.test import (
    test_inactive_tracks,
    test_neural_message_passing_1,
    test_neural_message_passing_2,
    test_mpn_tracking_forward_pass,
    test_longterm_hungarian_tracking_ioureid,
    test_mpn_tracking,
    val_mpn_tracking,
    run_tracker
)

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Exercise Part I - Long-Term ReID Tracker
The tracker `Hungarian_TrackerIoUReID` in `exercise_code/model/hungarian_tracker.py` has an obvious limitation: whenever we can not match a track with the detections of a given frame, the track will be killed (line 54). Therefore, if our detector misses an object in a single frame (due to, e.g. occlusion), we will not be able to recover that track, and we will start a new one. 

We want to allow the tracker to maintain unmatched tracks to fix this issue and refer to these tracks as inactive. During data association, we try to match the detected boxes for the current frame to the active and inactive tracks. Therefore, if a detector misses an object in a frame and the object reappears after a few frames, we try to match it to its corresponding track instead of creating a new one.

We use the `inactive` attribute from the `track` class in `exercise_code/model/tracker.py` to adapt our tracker. This attribute is a counter and shows how many frames a track has remained unmatched. Whenever we can match the track, we will set `inactive=0`; otherwise, we increase the counter. 

<div class="alert alert-info">
    <h3>Task: Inactive tracks</h3>
    <p> Go to <code>exercise_code/hungarian_tracker.py</code> and the <code>update_tracks</code> method. You must update the <code>inactive</code> attribute for the unmatched tracks. Also, remove those tracks where the <code>inactive</code> counter is bigger than the <code>patience</code>.</p>
</div>

<div class="alert alert-danger">
    <h3>Test: Inactive tracks</h3>
    <p> Run the following cell to execute the test case for the inactive tracker.</p>
</div>

In [4]:
_ = test_inactive_tracks()

Congratulations: No tracks have been removed before patience was reached.
Congratulations: No tracks have been removed before patience was reached. Counter is correct.
Congratulations: Tracks have been removed after patience was reached.
All tests of InactiveTracksTest passed. Tests passed: 3/3
Score: 100/100


If the previous test ran successfully you can now run the tracker on the validation sequence to evaluate its results.

In [5]:
old_tracker = False
if old_tracker:
    tracker = Hungarian_TrackerIoUReID()
else:
    tracker = Longterm_Hungarian_TrackerIoUReID(patience=10)

train_db = torch.load(dataset_dir.joinpath('reid_gnn', 'preprocessed_data_train_2.pth'))
val_sequences = MOT16Sequences('MOT16-reid', root_dir = dataset_dir.joinpath('MOT16'), vis_threshold=0.)
results_seq = run_tracker(val_sequences, train_db, tracker, device=device)

Runtime for all sequences: 1.3 s.


In [6]:
torch.save(results_seq, "exercise_code/test/"+tracker.name+".pth")

<div class="alert alert-danger">
    <h3>Test: Inactive tracks performance</h3>
    <p> Run the following cell to execute the test case for the performance of the inactive tracker. The second row serves as a reference.</p>
</div>

In [7]:
_ = test_longterm_hungarian_tracking_ioureid()

              IDF1       IDP       IDR      Rcll      Prcn  GT  MT  PT  ML   FP    FN  IDs   FM      MOTA      MOTP   IDt    IDa  IDm
MOT16-02  0.457617  0.649832  0.353156  0.522469  0.961378  62  12  37  13  390  8873  133  210  0.494322  0.090339  17.0  119.0  5.0
MOT16-02  0.457617  0.649832  0.353156  0.522469  0.961378  62  12  37  13  390  8873  133  210  0.494322  0.090339   NaN    NaN  NaN
Congratulations: Longterm_Hungarian_TrackerIoUReID seems to be correct for sequence MOT16-02 based on the metrics: mota idf1.
              IDF1       IDP       IDR      Rcll      Prcn   GT  MT  PT  ML   FP    FN  IDs   FM      MOTA      MOTP   IDt    IDa   IDm
MOT16-05  0.583493  0.690091  0.505421  0.688304  0.939795  133  54  67  12  305  2156  289  149  0.602429  0.141942  71.0  226.0  13.0
MOT16-05  0.583493  0.690091  0.505421  0.688304  0.939795  133  54  67  12  305  2156  289  149  0.602429  0.141942   NaN    NaN   NaN
Congratulations: Longterm_Hungarian_TrackerIoUReID seems to be c

## Part II - Building a tracker based on Neural Message Passing

Our ``Longterm_Hungarian_TrackerIoUReID`` is still limited when compared to current modern trackers. One disadvantage is that our tracker can only account for pairwise similarities among objects. Ideally, we would like it also to consider higher-order information.

To address this limitation, we will now build a tracker combining appearance and position information with a Message Passing Neural Network, inspired by the approach presented in [0].

The idea is to build a bipartite graph containing two sets of nodes for every tracking step: past tracks $A$ and detections $B$ in the current frame, and our set of edges will be $A\times B$. That is, we will connect every past track with every detection.
We will have initial node features (i.e. reid embeddings) matrices: $X_A \in \R^{|A| \times \text{node\_dim}}$ and $X_B \in \R^{|B|\times \text{node\_dim}}$ and an initial edge features tensor $E\in \R^{|A| \times |B| \times \text{edge\_dim}}$.
That means its $(i, j)$-th entry contains the edge features between node $i$ in $A$ and node $j$ in $B$.

We will use an MPN to refine these edge embeddings. For this, we will first build a Neural Message Passing layer based on the Graph Networks framework introduced in [1], as explained in the *A More General Framework* slides of [Lecture 5](https://youtu.be/BR3Y5bAz5Dw) (slides 70 to 75) of the old lecture recordings.

With the given layer, we will produce new node feature matrices $X_A'$ and $X_B'$ and edge features $E'$ with the same dimensionality.
The learning task will be to classify the edge embeddings in this graph, which is equivalent to predicting the entries of our data association similarity matrix. 

[0] [Learning a Neural Solver for Multiple Object Tracking](https://arxiv.org/abs/1912.07515)

[1] [Relational inductive biases, deep learning, and graph networks](https://arxiv.org/abs/1806.01261)

[2] [Brasó and Leal-Taixé, Learning a Neural Solver for Multiple Object Tracking, CVPR 2020](https://arxiv.org/abs/1912.07515)

[3] [Battaglia et al., Relational inductive biases, deep learning, and graph networks, arXiv 2018](https://arxiv.org/abs/1806.01261)



<div class="alert alert-info">
    <h3>Task: Neural Message Passing I</h3>
    <p> You have to implement a part of the neural message passing layer by adding the node and edge update steps. Go to <code>exercise_code/model/assign_net.py</code> and the class <code>BipartiteNeuralMessagePassingLayer</code>. There you need to complete the methods <code>edge_update(self, edge_embeds, nodes_a_embeds, nodes_b_embeds)</code> and <code>node_update(self, edge_embeds, nodes_a_embeds, nodes_b_embeds)</code>.
    <br>
    Note: You do not need to care about batching several graphs. This implementation will only work with a single graph at a time.
    </p>
</div>

In [8]:
_ = test_neural_message_passing_1()

Congratulations: The shape of the Edge Update seems to be correct
Congratulations: The shape of the Node Update seems to be correct
Congratulations: The input to the node MLP seems to be correct
Congratulations: The input to the edge MLP seems to be correct
All tests of NeuralMessagePassing passed. Tests passed: 4/4
Score: 100/100


## Building the entire network to predict similarities
We now build the network that generates initial node and edge features, performs neural message passing, and classifies edges in order to produce the final costs that we will use for data association.

[2] proposes, given two bounding boxes $(x_i, y_i, w_i, h_i)$ and  $(x_j, y_j, w_j, h_j)$ and timestamps $t_i$ and $t_j$, to compute an initial 5-dimensional edge feature vector as:
$$ E_{(i, j)} = \left(\frac{2(x_j - x_i)}{h_i + h_j}, \frac{2(y_j - y_i)}{h_i + h_j}, \log{\frac{h_i}{h_j}}, \log{\frac{w_i}{w_j}}, t_j - t_i \right )$$

<div class="alert alert-info">
    <h3>Task: Neural Message Passing II</h3>
    <p> Go to <code>exercise_code/model/message_passing_network.py</code> and the class <code>AssignmentSimilarityNet</code>. There you need to complete the methods <code>compute_motion_edge_feats(self, track_coords, current_coords, track_t, curr_t)</code> method. Previously, we used the IoU between predictions and detections as motion prior. Now, you can use the edge feature proposed in [1] as detailed in the cell above.
    <br>
    Feel free to engineer your own features (e.g. use IoU, etc.)</p>
</div>

<div class="alert alert-danger">
    <h3>Test: Neural Message Passing</h3>
    <p> Run the following cell to execute the test case for the second task of regarding neural message passing. This test will check the correct implementation of the given feature vector. If you implemented your own feature vector, this test will likely fail. However, you can adjust the feature dimension accordingly to pass the shape test. </p>
</div>

In [9]:
_ = test_neural_message_passing_2()

Congratulations: The shape of the Edge Initialization seems to be correct
All tests of NeuralMessagePassing passed. Tests passed: 1/1
Score: 100/100


## Putting everything together
Finally, we test our ``AssignmentSimilarityNet``, which will output a refined similarity matrix, which we can use for linear assignment. The forward pass through the assignment net will initialize the edge and node features and run the specified number of update iterations. The output is then a tensor of similarity matrices whose first dimension is the number of update steps.


Finally, we can integrate the AssignmentNet into our Tracker. We can keep everything as in ``LongTermReIDHungarianTracker`` except for the distance computation, which is now directly obtained via a forward pass through AssignmentSimilarityNet. 

<div class="alert alert-info">
    <h3>Task: Doing a Forward Pass</h3>
    <p> Go to <code>exercise_code/model/message_passing_network.py</code> and the class <code>MPN_Tracker</code>. There you need perform a forward pass through the <code>AssignmenSimilarityNet</code>.
</div>

<div class="alert alert-danger">
    <h3>Test: Doing a Forward Pass</h3>
    <p> Run the following cell to execute the test case for running a forward pass.</p>
</div>

In [10]:
_ = test_mpn_tracking_forward_pass()

Congratulations: The forward pass seems to work
All tests of ForwardPassTest passed. Tests passed: 1/1
Score: 100/100


## Training and evaluating our model

We provide all boilerplate code for training and evaluating our neural message passing tracker. Under the hood, we randomly sample boxes of frames from our training sequences to generate $A$. Then, we add sampled boxes from past frames to generate our second set $B$. Check out `LongTrackTrainingDataset` for details.

We train the model with a weighted cross-entropy loss to account for the class imbalance and add a weight $w>1$ for the positive samples in the binary cross-entropy loss. Check out `exercise_code/model/trainer.py`.
$$
\sum_{{i,j}} w \cdot y_{i,j} \log\left(\hat{y}_{i,j} \right) + \left(1-y_{i,j}\right)\log\left(1- \hat{y}_{i,j}\right),
$$ where $$y_{i,j} \in\{ 0,1\}$$ is one, if the edge is a match in the training data and zero, otherwise. The variable $$\hat{y_{i,j}} \in[0,1]$$ denotes the learned output probability of the edge being active, which we use for classification. No need to write any code from your side here!

Expert Knowledge: The binary cross entropy loss is minimizing the likelihood of statistically independent Bernoulli Experiments with success probability $\hat{y}_{i,j}\in[0,1]$
$$
\operatorname{arg}\min \prod_{i,j} \hat{y}_{i,j}^{y_{i,j}} +  (1-\hat{y}_{i,j})^{(1-{y_{i,j}})}.
$$
How is the weighted cross-entropy loss related to unbalanced data?
It penalizes more positive labels that are falsely classified, hence helping increase recall; you can expect your ML model to become more sensitive in correctly classifying positive cases out of all actual positives.
(Note: Recall=TP/P=TP/(TP+FN))

In [30]:
# Define our model, and init 
assign_net = AssignmentSimilarityNet(reid_network=None, # Not needed since we work with precomputed features
                                     node_dim=32, 
                                     edge_dim=64, 
                                     reid_dim=512, 
                                     edges_in_dim=6, 
                                     num_steps=10).to(device)

We only leave two sequences for validation in order to maximize the amount of training data.

In [31]:
MAX_PATIENCE = 20
train_db = torch.load(dataset_dir.joinpath('reid_gnn', 'preprocessed_data_train_2.pth'))
# We only keep two sequences for validation
dataset = LongTrackTrainingDataset(dataset='MOT16-train_wo_val', 
                                   db=train_db, 
                                   root_dir=dataset_dir.joinpath('MOT16'),
                                   max_past_frames=MAX_PATIENCE,
                                   vis_threshold=0.25)
data_loader = DataLoader(dataset, 
                        batch_size=20, 
                        collate_fn=lambda x: x,
                        shuffle=True,
                        num_workers=2,
                        drop_last=True)
dataset_val = LongTrackTrainingDataset(dataset='MOT16-val', 
                                   db=train_db, 
                                   root_dir=dataset_dir.joinpath('MOT16'),
                                   max_past_frames=MAX_PATIENCE,
                                   vis_threshold=0.25)
data_loader_val = DataLoader(dataset_val, 
                        batch_size=20, 
                        collate_fn=lambda x: x,
                        shuffle=True,
                        num_workers=2,
                        drop_last=True)
val_sequences = MOT16Sequences('MOT16-val', 
                        dataset_dir.joinpath('MOT16'),
                        vis_threshold=0.25)

### Let's start training!

Note that we have observed quite a lot of noise in validation scores among epochs and runs. That is due to the small size of our training and validation sets. Therefore, we perform early stopping to obtain the best-performing model on validation. In addition, changing the experiment seed and/ or relaunching the training might help if you suspect noise influencing your scores. 

In [32]:
# Init the optimizer and lr scheduler
optimizer = torch.optim.Adam(assign_net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5)
MAX_EPOCHS = 10 # adjust this for longer training, if you have problems with your GPU, try to train only for one epoch and continue with the rest of the exercise
EVAL_FREQ = 1

score_best = 0
for epoch in range(1, MAX_EPOCHS + 1):
    print(f"-------- EPOCH {epoch:2d} --------")
    train_assign_net_one_epoch(model=assign_net, data_loader=data_loader, optimizer=optimizer, print_freq=100)
    scheduler.step()

    if epoch % EVAL_FREQ == 0:
        print(f"-------- Validation --------")
        # validation on long tracking dataset
        metrics_accum = evaluate_assign_net(model=assign_net.eval(), data_loader=data_loader_val)
        # validation using the tracker
        tracker = MPN_Tracker(assign_net=assign_net.eval(), patience=MAX_PATIENCE)
        results_seq = run_tracker(val_sequences, db=train_db, tracker=tracker, device=device)
        torch.save(results_seq, "exercise_code/test/"+tracker.name+".pth")
        score = val_mpn_tracking()
        if score>score_best:
            score_best=score
            torch.save(assign_net, output_dir.joinpath("assign_net.pth"))


-------- EPOCH  1 --------
cuda:0


0it [00:00, ?it/s]

100it [00:23,  4.46it/s]

Iter 100. Loss: 12.337. Accuracy: 0.911. Recall: 0.665. Precision: 0.385


111it [00:25,  4.33it/s]

...

Iter 100. Loss: 10.516. Accuracy: 0.998. Recall: 0.997. Precision: 0.979


111it [00:25,  4.37it/s]

-------- Validation --------



74it [00:08,  9.14it/s]


Iter 74. Loss: 857.888. Accuracy: 0.999. Recall: 0.997. Precision: 0.983
Runtime for all sequences: 5.6 s.
              IDF1
MOT16-02  0.469472
MOT16-11  0.673076
Your tracker MPN_Tracker reached the mean idf1 0.57 on sequence MOT16-val.
Test passed 57/100


In [33]:
torch.save(assign_net, output_dir.joinpath("assign_net.pth"))

In [34]:
assign_net = torch.load( output_dir.joinpath("assign_net.pth"))
tracker =  MPN_Tracker(assign_net=assign_net.eval(), patience=MAX_PATIENCE)
val_sequences = MOT16Sequences('MOT16-val', 
                        dataset_dir.joinpath('MOT16'),
                        vis_threshold=0.25)
results_seq = run_tracker(val_sequences, train_db, tracker, device=device)

Runtime for all sequences: 5.5 s.


In [35]:
torch.save(results_seq, "exercise_code/test/"+tracker.name+".pth")
_ = test_mpn_tracking()

              IDF1       IDP       IDR      Rcll      Prcn  GT  MT  PT  ML   FP    FN  IDs   FM      MOTA      MOTP   IDt   IDa  IDm
MOT16-02  0.469472  0.666667  0.362306  0.522469  0.961378  62  11  38  13  390  8873  135  220  0.494215  0.094643  37.0  95.0  6.0
MOT16-02  0.457617  0.649832  0.353156  0.522469  0.961378  62  12  37  13  390  8873  133  210  0.494322  0.090339   NaN   NaN  NaN
Congratulations: MPN_Tracker seems to be correct for sequence MOT16-02 based on the metrics: mota idf1.
              IDF1       IDP       IDR      Rcll      Prcn  GT  MT  PT  ML   FP    FN  IDs  FM      MOTA      MOTP   IDt   IDa  IDm
MOT16-11  0.673076  0.742051  0.615833  0.801717  0.966032  75  44  24   7  266  1871   53  88  0.767910  0.082431  14.0  35.0  4.0
MOT16-11  0.650489  0.717150  0.595167  0.801717  0.966032  75  44  24   7  266  1871  143  91  0.758372  0.082736   NaN   NaN  NaN
Congratulations: MPN_Tracker seems to be correct for sequence MOT16-11 based on the metrics: mota idf

# Exercise submission

In [36]:
from exercise_code.submit import submit_exercise

submit_exercise('../output/exercise03')

relevant folders: ['models', 'exercise_code']
notebooks files: ['reid_gnn.ipynb']
Adding folder models
Adding folder exercise_code
Adding notebook reid_gnn.ipynb
Zipping successful! Zip is stored under: /home2/yehia.ahmed/cv3/cv3dst/output/exercise03.zip
